该笔记本记载了gee上通过样本点获取训练数据-训练回归模型-云端调用-回归显示的全流程

In [1]:
import ee
import geemap
import pandas as pd
from geemap import ml
from sklearn import ensemble
geemap.set_proxy(port=4780)
Map = geemap.Map(center=(30, 120), zoom=4)

合成的土地利用poi点和边界导入

In [2]:
points = ee.FeatureCollection('projects/assimov/assets/wuhanpoi')
ROI = ee.FeatureCollection("users/yamiletsharon250/wuhan").geometry()
Map.addLayer(ROI, {}, "武汉")
Map.addLayer(points, {}, "武汉点")
Map

Map(center=[30, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

常规的去云以及波段合成操作

In [3]:
def rmL8Cloud(image):
   cloudShadowBitMask = (1 << 3); 
   cloudsBitMask = (1 << 5); 
   qa = image.select('pixel_qa'); 
   mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0)); 
   return image.updateMask(mask).toFloat().divide(10000).copyProperties(image).copyProperties(image, ["system:time_start",'system:time_end','system:footprint'])

collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
                  .map(rmL8Cloud)\
                  .filterDate('2021-01-01','2022-01-01') \
                  .filterBounds(ROI) \
                  .filter(ee.Filter.lte('CLOUD_COVER',5))\
                  
image=collection.mean().clip(ROI)
def NDWI(img):
  nir = img.select('B4')
  green = img.select('B2')
  ndwi = img.expression(
    '(B3-B5)/(B3+B5)',
    {
      'B5':nir,
      'B3':green
    })
  return ndwi

def EWI(img):
  swir1 = img.select('B6')
  nir = img.select('B5')
  green = img.select('B3')
  ewi = img.expression(
    '(B3-B5-B6)/(B3+B5+B6)',
    {
      'B6':swir1,
      'B5':nir,
      'B3':green
    })
  return ewi

MNDWI = image.normalizedDifference(['B3', 'B6']).rename('MNDWI');#计算MNDWI
ndbi = image.normalizedDifference(['B6', 'B5']).rename('NDBI');#计算NDBI
ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');#计算NDVI
ewi = EWI(image).rename('EWI');#计算NDVI
ndwi = NDWI(image).rename('NDWI');#计算MNDWI
lswi = image.normalizedDifference(['B5','B6']).rename('LSWI')
nbr2 = image.normalizedDifference(["B6", "B7"]).rename("NBR2")
awei = image.expression(
              '4*(green-SWIR1)-(0.25*NIR+2.75*SWIR2)',{
              'green':image.select('B3'),
              'NIR':image.select('B5'),
              'SWIR1':image.select('B6'),
              'SWIR2':image.select('B7'),
            }).float().rename('AWEI')
image=image \
      .addBands(ndvi) \
      .addBands(ndbi) \
      .addBands(MNDWI) \
      .addBands(ndwi) \
      .addBands(ewi) \
      .addBands(lswi) \
      .addBands(awei) \
      .addBands(nbr2)

设置训练波段，导出训练数据至本地

In [6]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','MNDWI','NDBI','NDVI','EWI','NDWI','AWEI','LSWI',"NBR2"]
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': ['landcover'],
  'scale': 30
})
geemap.ee_to_csv(training,"rftrain.csv")

可视化dataframe

In [5]:
df = pd.read_csv("rftrain.csv")
df

,system:index,landcover,AWEI,NDWI,NDVI,B2,NBR2,B3,B4,B5,B6,MNDWI,B7,EWI,NDBI,LSWI
0,00000000000000000087_0,1,0.400050,-0.222047,-0.578305,0.0741,0.229167,0.1104,0.1164,0.0311,0.0059,0.898538,0.0037,0.497965,-0.681081,0.681081
1,00000000000000000087_1,1,0.402425,-0.223677,-0.572005,0.0741,0.239130,0.1107,0.1168,0.0318,0.0057,0.902062,0.0035,0.493927,-0.696000,0.696000
2,00000000000000000087_2,1,0.402250,-0.229963,-0.589709,0.0735,0.212766,0.1107,0.1174,0.0303,0.0057,0.902062,0.0037,0.509202,-0.683333,0.683333
3,00000000000000000087_3,1,0.406175,-0.231491,-0.592717,0.0737,0.258427,0.1113,0.1181,0.0302,0.0056,0.904192,0.0033,0.513256,-0.687151,0.687151
4,00000000000000000087_4,1,0.405400,-0.227723,-0.592968,0.0741,0.266667,0.1112,0.1178,0.0301,0.0057,0.902481,0.0033,0.512925,-0.681564,0.681564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,00000000000000000588_0,5,-0.991900,-0.152518,0.539523,0.0589,0.226929,0.0822,0.0801,0.2678,0.2187,-0.453639,0.1378,-0.710920,-0.100925,0.100925
3588,0000000000000000058a_0,5,-1.202575,-0.170856,0.124947,0.1468,0.052651,0.1869,0.2073,0.2665,0.2909,-0.217664,0.2618,-0.497783,0.043775,-0.043775
3589,0000000000000000058a_1,5,-1.201125,-0.174100,0.112510,0.1537,0.048530,0.1975,0.2185,0.2739,0.2960,-0.199595,0.2686,-0.485275,0.038779,-0.038779
3590,0000000000000000058d_0,5,-0.970475,-0.211885,0.165001,0.1114,0.084607,0.1481,0.1713,0.2390,0.2378,-0.232444,0.2007,-0.526004,-0.002517,0.002517


随机森林回归训练n_trees为参数，还有其他参数可以调，我只是简单打个样

In [6]:
feature_names = ['AWEI','NDWI','NDVI','B2','NBR2','B3','B4','B5','B6','MNDWI','B7','EWI','NDBI','LSWI']
label = "landcover"
X = df[feature_names]
y = df[label]
n_trees = 10
rf = ensemble.RandomForestRegressor(n_trees).fit(X, y)
rf

将rf模型转为trees

In [9]:
trees = ml.rf_to_strings(rf, feature_names,output_mode='REGRESSION')

trees转classifier

In [11]:
ee_classifier = ml.strings_to_classifier(trees)
#ee_classifier.getInfo()

将模型保存到本地

In [4]:
ml.trees_to_csv(trees, "trees.csv")
ee_classifierlocal =ml.csv_to_classifier("trees.csv")
#ee_classifierlocal.getInfo()

将模型保存到云端，可能得等会，因为保存要时间，上传有限制不能超过40m

In [13]:
ml.export_trees_to_fc(trees, 'users/yamiletsharon250/rfregression')

In [ ]:
rf_fc = ee.FeatureCollection('users/yamiletsharon250/rfregression')
ee_classifiercloud = ml.fc_to_classifier(rf_fc)
#ee_classifiercloud.getInfo()

回归模型验证，使用的是local模型，cloud模型在js端验证

In [7]:
regression = image.select(bands).classify(ee_classifierlocal, 'predicted')

In [8]:
regressionMin = (regression.reduceRegion(
    **{
'reducer': ee.Reducer.min(),
'scale': 30,
'crs': 'EPSG:3857',
'bestEffort': True,
'tileScale': 16
}
))
regressionMax = (regression.reduceRegion(
    **{
'reducer': ee.Reducer.max(),
'scale': 30,
'crs': 'EPSG:3857',
'bestEffort': True,
'tileScale': 16
}
))

In [18]:

# regressionMin.getNumber('predicted').getInfo()
# regressionMax.getNumber('predicted').getInfo()

1

In [10]:
viz = {min: 1,max: 5,'opacity':1,'palette':['blue','yellow','ffffff','red','green']}
Map.addLayer(regression, viz, 'Regression')
Map

Map(bottom=2097.0, center=[21.53484700204879, 119.00390625000001], controls=(WidgetControl(options=['position'…